<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Graph_Neural_Networks_(GNNs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch_geometric

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv

# Define the SimpleGNN class
class SimpleGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)  # First GCN layer
        self.conv2 = GCNConv(hidden_dim, output_dim)  # Second GCN layer

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index))  # Apply ReLU activation after the first layer
        return self.conv2(x, edge_index)  # Second GCN layer

# Example usage
model = SimpleGNN(input_dim=3, hidden_dim=16, output_dim=2)  # Instantiate the GNN model
x = torch.randn((4, 3))  # Node features for 4 nodes (num_nodes=4, feature_dim=3)
edge_index = torch.tensor([[0, 1, 2, 3], [1, 0, 3, 2]], dtype=torch.long)  # Edge connections
output = model(x, edge_index)

# Print the shape of the output
print(output.shape)  # Expected shape: [4, 2] (num_nodes, output_dim)